# Quantum Error Correction

> My first time trying `qiskit` and `jupyter`...

In this document, I'd like to construct and simulate a very simple single-qubit error correction code, mostly to familiarize myself with `qiskit` and `jupyter`. We will start by defining the type of error that we wish to correct: a single-qubit bitflip. We will construct a function that does this randomly to one of the physical qubits in a given logical qubit:

In [1]:
import numpy as np

# randomly bitflip one of the physical qubits of 'q'
def single_qubit_error(q, circ):
    switch = q[int(np.random.choice(range(len(q))))]
    circ.x(switch)

Next, we'll extend `qiskit.QuantumCircuit` to define some gates that `qiskit` doesn't include by default, particularly the Toffoli gate. We'll do this with a partially applied function implementations because I have a soft spot for functional programming.

In [2]:
# do a Toffoli gate on q3, conditioned on q1 and q2
def toffoli(q1, q2, q3, circ):
    circ.h(q3)
    circ.cx(q2, q3)
    circ.tdg(q3)
    circ.cx(q1, q3)
    circ.t(q3)
    circ.cx(q2, q3)
    circ.tdg(q3)
    circ.cx(q1, q3)
    circ.t(q2)
    circ.t(q3)
    circ.h(q3)
    circ.cx(q1, q2)
    circ.t(q1)
    circ.tdg(q2)
    circ.cx(q1, q2)

Now let's put together the error correction circuit! First, we'll perform a parity check by reading single-qubit errors on the logical qubit into the ancillary qubits:

In [3]:
# read error in circuit 'circ' from qubits 'q'
# onto ancillary 'a'
def read_error(q, a, circ):
    circ.cx(q[0], a[0])
    circ.cx(q[1], a[0])
    circ.cx(q[0], a[1])
    circ.cx(q[2], a[1]);

Then we'll correct any errors in the logical qubit based on the values of the ancillary bits:

In [4]:
# correct a single parity error in circuit 'circ'
# with data on 'q' and ancillary on 'a'
def correct_parity(q, a, circ):
    circ.toffoli(a[0], a[1], q[0])
    circ.x(a[0])
    circ.toffoli(a[0], a[1], q[2])
    circ.x(a[0])
    circ.x(a[1])
    circ.toffoli(a[0], a[1], q[1]);

Finally, we can put this together into an experimental run. Each new trial, we'll build a new circuit, introducing some random single-qubit error to see if it is properly detected and corrected:

In [5]:
import qiskit
from functools import partial

def build_circuit():
    # setup registers: logical qubit, ancillary qubits, classical readout
    # important: qubits are initialized in the '0' state
    qubit = qiskit.QuantumRegister(3, 'q')
    ancillary = qiskit.QuantumRegister(2, 'a')
    readout = qiskit.ClassicalRegister(3, 'c')
    
    # init circuit and add custom methods
    circuit = qiskit.QuantumCircuit(qubit, ancillary, readout)
    circuit.single_qubit_error = partial(single_qubit_error, circ=circuit)
    circuit.toffoli = partial(toffoli, circ=circuit)
    circuit.read_error = partial(read_error, circ=circuit)
    circuit.correct_parity = partial(correct_parity, circ=circuit)
    
    # introduce and correct a random error
    circuit.single_qubit_error(qubit)
    circuit.read_error(qubit, ancillary)
    circuit.correct_parity(qubit, ancillary)
    
    # measure qubits to verify results
    circuit.measure(qubit, readout)
    return circuit

Now, we'll run the trial on the `qiskit` Aer simulator and display the aggregate results:

In [6]:
from collections import Counter

# simulate a single circuit
def simulate_circuit(circuit):
    backend = qiskit.BasicAer.get_backend('qasm_simulator')
    job = qiskit.execute(circuit, backend, shots=1)
    return Counter(job.result().get_counts(circuit))

# display results of N runs
N = 100
sum([simulate_circuit(build_circuit()) for _ in range(N)], Counter())

Counter({'000': 100})

As we would expect, the logical qubit remains in the `'000'` state every single time, indicating that our error correction was successful.